In [1]:
!pip install h5py numpy


In [2]:
import h5py
import numpy as np

# Load the datasets (ensure the files are in the same directory as the notebook)
train_dataset = h5py.File('Train.h5', "r")
test_dataset = h5py.File('Test.h5', "r")

# Inspect the keys in the train and test datasets
print("Train dataset keys:", list(train_dataset.keys()))
print("Test dataset keys:", list(test_dataset.keys()))


Train dataset keys: ['images', 'labels']
Test dataset keys: ['images', 'labels']


In [3]:
# Use the correct keys based on your inspection
X_train = np.array(train_dataset['images'][:])
Y_train = np.array(train_dataset['labels'][:])
X_test = np.array(test_dataset['images'][:])
Y_test = np.array(test_dataset['labels'][:])

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to one-hot encoding
Y_train = np.eye(np.max(Y_train) + 1)[Y_train.reshape(-1)]
Y_test = np.eye(np.max(Y_test) + 1)[Y_test.reshape(-1)]

print(f"Training data shape: {X_train.shape}, {Y_train.shape}")
print(f"Test data shape: {X_test.shape}, {Y_test.shape}")


Training data shape: (2626, 128, 128, 3), (2626, 5)
Test data shape: (120, 128, 128, 3), (120, 5)


In [4]:
def softmax(Z):

    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / exp_Z.sum(axis=1, keepdims=True)


In [5]:
def compute_cost(Y, Y_hat):

    m = Y.shape[0]
    cost = -np.sum(Y * np.log(Y_hat + 1e-8)) / m
    return cost


In [6]:
def gradient_descent(X, Y, learning_rate=0.01, epochs=1000):

    m, h, w, c = X.shape
    n = h * w * c  # Flattened feature size
    X_flat = X.reshape(m, n)  # Flatten images

    k = Y.shape[1]  # number of classes
    W = np.zeros((n, k))
    b = np.zeros((1, k))
    
    for i in range(epochs):
        # Forward propagation
        Z = np.dot(X_flat, W) + b
        Y_hat = softmax(Z)
        
        # Compute cost
        cost = compute_cost(Y, Y_hat)
        
        # Backward propagation (calculating gradients)
        dW = np.dot(X_flat.T, (Y_hat - Y)) / m
        db = np.sum(Y_hat - Y, axis=0, keepdims=True) / m
        
        # Update parameters
        W -= learning_rate * dW
        b -= learning_rate * db
        
        # Print cost every 100 epochs
        if i % 100 == 0:
            print(f"Cost after epoch {i}: {cost}")
    
    return W, b


In [ ]:
def predict(X, W, b):

    m, h, w, c = X.shape
    n = h * w * c
    X_flat = X.reshape(m, n)  # Flatten images
    
    Z = np.dot(X_flat, W) + b
    Y_hat = softmax(Z)
    return np.argmax(Y_hat, axis=1)

# Train the model
W, b = gradient_descent(X_train, Y_train, learning_rate=0.01, epochs=1000)

# Predict on the test set
predictions = predict(X_test, W, b)

# Calculate accuracy
Y_test_flat = np.argmax(Y_test, axis=1)  # Flatten test labels to match predictions
accuracy = np.mean(predictions == Y_test_flat)
print(f"Test set accuracy: {accuracy * 100:.2f}%")


Cost after epoch 0: 1.609437862434102
